In [1]:
import duckdb
import os

In [2]:
con = duckdb.connect(database = "cicids2017.duckdb", read_only = False)

In [3]:
con.execute('CREATE TABLE flows AS SELECT "Flow ID", Label FROM read_csv_auto("../datasets/CICIDS2017_improved/*.csv");')

In [4]:
con.execute("UPDATE Flows SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [5]:
con.execute('SELECT DISTINCT Label FROM flows')
Labels = con.fetch_df()
Labels

,Label
0,Infiltration - Portscan
1,Web Attack - XSS
2,Web Attack - Brute Force
3,FTP-Patator
4,Portscan
5,DDoS
6,BENIGN
7,DoS Slowhttptest
8,DoS Slowloris
9,Infiltration


In [6]:
con.execute('SELECT Label, COUNT(*) AS count FROM Flows GROUP BY Label')
counted_labels = con.fetch_df()
counted_labels

,Label,count
0,Infiltration - Portscan,71767
1,Web Attack - XSS,18
2,Portscan,159066
3,DDoS,95144
4,FTP-Patator,3972
5,Web Attack - Brute Force,73
6,BENIGN,1594545
7,DoS Slowhttptest,1740
8,DoS Slowloris,3859
9,DoS GoldenEye,7567


In [7]:
con.execute('CREATE TABLE sampled_flows ("Flow ID" VARCHAR)')

In [8]:
con.execute('DELETE FROM sampled_flows')

for label in Labels.to_numpy():
    con.execute('INSERT INTO sampled_flows SELECT "Flow ID" FROM flows WHERE Label = \'{}\' ORDER BY RANDOM() LIMIT 2000'.format(label[0]))

In [14]:
con.execute('CREATE TABLE CICIDS2017_SAMPLED AS SELECT * EXCLUDE (id,"Flow ID","Src IP","Src Port","Dst IP","Dst Port",Timestamp) FROM read_csv_auto("../datasets/CICIDS2017_improved/*.csv") WHERE "Flow ID" IN (SELECT * FROM sampled_flows)')

In [15]:
con.execute("UPDATE CICIDS2017_SAMPLED SET Label = regexp_replace(Label, '(.+) - Attempted','BENIGN')")

In [16]:
con.execute("COPY (SELECT * FROM CICIDS2017_SAMPLED) TO '../datasets/CICIDS2017_improved.parquet' (FORMAT PARQUET)")